In [11]:
pip install ray[default] torch numpy wandb tqdm gymnasium lz4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.7 MB/s eta 0:00:00


In [12]:
import numpy as np
import gym
from gym import spaces
from ray.rllib.env.multi_agent_env import MultiAgentEnv

class CichyEnv(MultiAgentEnv):
    def __init__(self, images, expert_rdms):
        self.images = images
        self.expert_rdms = expert_rdms
        self.num_agents = 2
        self.agent_ids = ["IT", "EVC"]


        self.observation_space = spaces.Dict({
            "image": spaces.Box(low=0, high=255, shape=(224, 224, 3), dtype=np.uint8),
            "other_action": spaces.Box(low=-1, high=1, shape=(1,), dtype=np.float32)
        })

        # Continuous: 1. expending activity units 2. sending signal
        self.action_space = spaces.Box(low=-1, high=1, shape=(2,), dtype=np.float32)

    def reset(self):
        self.current_step = 0
        self.state = {agent: {"image": self.images[self.current_step], "other_action": np.array([0.0])} for agent in self.agent_ids}
        return self.state

    def step(self, action_dict):
        obs, rewards, dones, infos = {}, {}, {}, {}
        self.current_step += 1

        for agent_id in self.agent_ids:
            other_agent_id = "EVC" if agent_id == "IT" else "IT"
            obs[agent_id] = {"image": self.images[self.current_step], "other_action": action_dict[other_agent_id][1]}
            rewards[agent_id] = self._calculate_reward(agent_id, action_dict[agent_id])
            dones[agent_id] = self.current_step >= len(self.images) - 1
            infos[agent_id] = {}

        dones["__all__"] = all(dones.values())
        return obs, rewards, dones, infos

    def _calculate_reward(self, agent_id, action):
      ## placeholder now,
      ## thinking of BIC
        return -np.sum((self.expert_rdms[agent_id][self.current_step] - action) ** 2)

